# Redfin Listing Price Predictor

## Instructions

Go To Redfin.com and find the listed property you want to predict 
- 1. Navigate to Cell in the Toolbar
- 2. Select Run All
- 3. Input House Information 
- 4. Copy and Paste House Description 
- 5. Input House Listing Price 
- 6. Scroll to the Bottom to Get Predictions

- PROPERTY TYPE INPUT OPTIONS (CASE SENSITIVE) 
    - Single Family Residential
    - Condo/Co-op	
    - Townhouse

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import spacy 
import re
from gensim.models.phrases import Phraser, Phrases
import pickle
from datetime import datetime
import time

In [2]:
current_year = datetime.now().year
current_month = datetime.now().month
current_quarter = (current_month - 1) // 3 + 1  # Calculate the quarter based on the current month
current_date = datetime.now().date()

columns = [
    "Year",
    "Quarter",
    "YEAR BUILT",
    "SALE TYPE",
    "SOLD DATE",
    "PROPERTY TYPE",
    "ZIP OR POSTAL CODE",
    "BEDS",
    "BATHS",
    "LOCATION",
    "SQUARE FEET",
    "LOT SIZE", 
    "Description"
]

df = pd.DataFrame(columns=columns)
for column in columns:
    if column == "Year":
        user_input = current_year
    elif column == "Quarter":
        user_input = current_quarter
    elif column == "SOLD DATE":
        user_input = current_date 
    elif column == "SALE TYPE":
        user_input = "PAST SALE"  
    else:
        user_input = input(f"Enter {column}: ")
    df.at[0, column] = user_input
price = input(f"Enter Price")


Enter YEAR BUILT: 2011
Enter PROPERTY TYPE: Townhouse
Enter ZIP OR POSTAL CODE: 98136
Enter BEDS: 3
Enter BATHS: 2.5
Enter LOCATION: Seaview
Enter SQUARE FEET: 1500
Enter LOT SIZE: 750
Enter Description: Welcome home! This contemporary 3 Bed 2.25 Bath Townhome, located in the heart of West Seattle, features elegant designer finishes & thoughtful floor plan. Spacious Kitchen boasts plank cabinets, Ceasar stone counters, stainless steel appliances, & connects to charming patio for summer BBQ's. Built green in 2011 with Solar Panels, this home blends eco-friendly design with modern aesthetics. The primary suite includes your own private retreat deck with sunset views & a humongous walk-in closet. Incredible proximity to Morgan Junction Restaurants & Bars, a block from Starbucks, the serenity of Lincoln Park & Alki Beach, Westwood Village shopping, & easy commute to downtown Seattle... this is the one!
Enter Price720000


In [3]:
df["Age"] = 2023 - df["YEAR BUILT"].astype("int")
df["Year"] = df["Year"].astype("int")
df["ZIP OR POSTAL CODE"] = df["ZIP OR POSTAL CODE"].astype("int")
df["BEDS"] = df["BEDS"].astype("float")
df["BATHS"] = df["BATHS"].astype("float")
df["SQUARE FEET"] = df["SQUARE FEET"].astype("int")
df["LOT SIZE"] = df["LOT SIZE"].astype("int")
df["Quarter"] = df["Quarter"].astype("int")
df["SOLD DATE"] = pd.to_datetime(df["SOLD DATE"])
df["ZIP OR POSTAL CODE"] = df["ZIP OR POSTAL CODE"].astype("int")

In [4]:
df.drop("YEAR BUILT", axis =1)

,Year,Quarter,SALE TYPE,SOLD DATE,PROPERTY TYPE,ZIP OR POSTAL CODE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,Description,Age
0,2023,4,PAST SALE,2023-11-16,Townhouse,98136,3.0,2.5,Seaview,1500,750,Welcome home! This contemporary 3 Bed 2.25 Bat...,12


In [5]:
df ["Description"] = df["Description"].str.lower()

In [6]:
nlp = spacy.load('en_core_web_sm')
def lemmatize(text): 
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

In [7]:
df["Description"] = df["Description"].apply(lemmatize)

In [8]:
stop_words = set(stopwords.words('english'))

In [9]:
df["Description"] = df["Description"].apply(word_tokenize)

In [10]:
clean_words = []
for tokenized_description in df["Description"]:
    cleaned_tokens = [token for token in tokenized_description if token not in stop_words]
    clean_words.append(cleaned_tokens)

In [11]:
df["Description"] = clean_words

In [12]:
def clean_tokens(tokens):
    cleaned_tokens = []
    for token in tokens:
        cleaned_token = re.sub(r'[^a-zA-Z0-9]', '', token)
        if cleaned_token:
            cleaned_tokens.append(cleaned_token)
    return cleaned_tokens



In [13]:
df["Description"] = df["Description"].apply(clean_tokens)

In [14]:
with open('Pickled Models/bigram_model.pkl', 'rb') as f:
    bigram = pickle.load(f)

In [15]:
df['Description'] = df['Description'].apply(lambda tokens: ' '.join(bigram[tokens]))

In [16]:
with open('Pickled Models/vectorizer.pkl', 'rb') as c:
    vectorizer = pickle.load(c)

In [17]:
transformed_df = vectorizer.transform(df["Description"])

In [18]:
df_bow = pd.DataFrame(transformed_df.toarray(), columns=vectorizer.get_feature_names_out())

In [19]:
df.reset_index(drop=True, inplace=True)
df_bow.reset_index(drop=True, inplace=True)

In [20]:
df_combined = pd.concat([df.drop('Description', axis=1), df_bow], axis=1)

In [21]:
df_combined = pd.concat([df.drop('Description', axis=1), df_bow], axis=1)

In [22]:
with open('Pickled Models/XGBPipeline.pkl', 'rb') as z:
    bestmodel = pickle.load(z)

In [23]:
prediction = bestmodel.predict(df_combined)

In [24]:
price = int(price)

In [25]:
prediction[0]

729865.9

In [26]:
percentage_error = ((price - prediction[0]) / price) * 100

In [27]:
print(f"Predicted Sale Price: ${prediction[0]:.0f}")
print(f"Listing vs. Predicted Sale Price Error: ${prediction[0] - price:.0f}")
print(f"Percentage Error: {percentage_error:.2f}%")

Predicted Sale Price: $729866
Listing vs. Predicted Sale Price Error: $9866
Percentage Error: -1.37%
